# Project 1 - Recommender system for 1700 random movies 

## Data and methods

Let us start by reading the data:

In [38]:
from pyspark.sql.session import SparkSession

In [39]:
spark = SparkSession.builder.getOrCreate()

In [40]:
data = spark.read.csv("ratings.csv", header='true').drop('timestamp')
data.show()

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|      2|   3.5|
|     1|     29|   3.5|
|     1|     32|   3.5|
|     1|     47|   3.5|
|     1|     50|   3.5|
|     1|    112|   3.5|
|     1|    151|   4.0|
|     1|    223|   4.0|
|     1|    253|   4.0|
|     1|    260|   4.0|
|     1|    293|   4.0|
|     1|    296|   4.0|
|     1|    318|   4.0|
|     1|    337|   3.5|
|     1|    367|   3.5|
|     1|    541|   4.0|
|     1|    589|   3.5|
|     1|    593|   3.5|
|     1|    653|   3.0|
|     1|    919|   3.5|
+------+-------+------+
only showing top 20 rows



How many entries are there in the data?

In [41]:
data.count()

20000263

In [42]:
data.describe("rating").show()

+-------+------------------+
|summary|            rating|
+-------+------------------+
|  count|          20000263|
|   mean|3.5255285642993797|
| stddev| 1.051988919294227|
|    min|               0.5|
|    max|               5.0|
+-------+------------------+



Converting the data into numeric values:

In [43]:
from pyspark.sql.types import DoubleType, IntegerType

data = data.withColumn("movieId", data["movieId"].cast(IntegerType()))
data = data.withColumn("userId", data["userId"].cast(IntegerType()))
data = data.withColumn("rating", data["rating"].cast(DoubleType()))
data.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)



## Sampling the data

In [44]:
import pandas as pd
data = pd.read_csv('ratings.csv').drop('timestamp', axis=1)

In [45]:
data.head()

,userId,movieId,rating
0,1,2,3.5
1,1,29,3.5
2,1,32,3.5
3,1,47,3.5
4,1,50,3.5


In order to train our models, we need to sample our data into a smaller dataset. 
<br> Let us start by selecting 3000 random movies:

In [46]:
import numpy as np
np.random.seed(1)
sample = data['movieId'].unique()
sample_movies = np.random.choice(sample, 3000)
data = data.loc[data['movieId'].isin(sample_movies)]

In [47]:
data.groupby('userId').nunique().count()

userId     137453
movieId    137453
rating     137453
dtype: int64

We end up with 3000 distinct movies and 137453 distinct users. 
<br>Let us randomly select 10,000 users to build our dataset.

In [48]:
sample = data['userId'].unique()
sample_users = np.random.choice(sample, 10000)
dataset = data.loc[data['userId'].isin(sample_users)]
dataset

,userId,movieId,rating
964,11,39,4.5
966,11,110,4.0
969,11,153,3.5
970,11,158,4.0
978,11,231,4.0
...,...,...,...
19991441,138424,480,5.0
19991460,138424,653,4.0
19991473,138424,1097,4.0
19991475,138424,1233,3.0


In [49]:
dataset.groupby('movieId').nunique().count()

userId     1724
movieId    1724
rating     1724
dtype: int64

In [50]:
dataset.groupby('userId').nunique().count()

userId     9667
movieId    9667
rating     9667
dtype: int64

Interesting, we only have 9,667 users instead of 10,000... But well, it will be enough for our study!
The number of movies has reduced to 1,724.

Let us now split our datset in a train set and a test set. We will use both of them for the rest of our study in this homework:

In [51]:
dataset = spark.createDataFrame(dataset)
(training, test) = dataset.randomSplit([0.8,0.2], seed=0)
print(training.count(), test.count())

142795 35679


## Statistics

We can make some statistics about our new set of movies to better understand it. Here is the number of ratings each movie has:

In [52]:
stat = dataset.groupBy("movieId").count().sort('count', ascending=False)
stat.describe('count').show()

+-------+------------------+
|summary|             count|
+-------+------------------+
|  count|              1724|
|   mean|103.52320185614849|
| stddev|318.52276197573184|
|    min|                 1|
|    max|              4172|
+-------+------------------+



Even though we kept a small part of the users, we still have at least 326 ratings per movie which is enough for our study.

Here we can see the number of ratings with respect to the number of movies:

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x1117c9090>]],
      dtype=object)

In [53]:
%matplotlib inline
import pandas as pd

panda_stat = stat.toPandas()
bin_ = [i*500 for i in range(10)]
panda_stat.hist(column='count', bins=bin_)

We can observe that the vast majority of the movies have less than 1,000 ratings and just a few of them have more ratings. This is the long tail effect: the popular movies (that represent a small proportion of the overall dataset) are far more rated than the other ones. 

# Metrics

In this section, we will define the metrics functions which we will use later to assess the performances of our models:
- MAE
- RMSE
- precision

MAE and RMSE:

In [54]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

def RMSE(y, y_predicted):
    return np.sqrt(mean_squared_error(y, y_predicted))

Precision:

In [55]:
# Let us first create a function to transform our raw predictions into appropriate ones.
def round_rating(x): 
    dec = x - int(x)
    if (dec >= 0.25) and (dec < 0.75):
        return int(x) + 0.5
    else:
        return int(x) + (dec > 0.5)

# Precision function metric:
def precision(y, y_predicted):
    y_predicted = [round_rating(k) for k in y_predicted]       
    TP = 0
    for i in range(len(y)):
        if y[i] == y_predicted[i]:
            TP += 1
    return TP/len(y)

## Baseline Model

We first need a baseline model to later compare the performances of our personalized models. We will use this one: 
<br><br>
$$r_{ui}=\mu$$
<br>
with $\mu$: global average rating

In [56]:
training.describe("rating").show()

+-------+-----------------+
|summary|           rating|
+-------+-----------------+
|  count|           142795|
|   mean|3.566161280156868|
| stddev|1.051374063014604|
|    min|              0.5|
|    max|              5.0|
+-------+-----------------+



In [57]:
mu = 3.639545494108101

In [58]:
def baseline_rating(userId, movieId):
    return mu

In [59]:
test_baseline = test.toPandas()

In [60]:
import time

t0 = time.time()
y_predicted = []
for i in range(len(test_baseline.index)):
    movieId = test_baseline.iloc[i, 1]
    userId = test_baseline.iloc[i, 0]
    y_predicted.append(baseline_rating(userId, movieId))
t_baseline = time.time() - t0
print ("Time of the training:", t_baseline)

Time of the training: 1.1117608547210693


In [61]:
y = test_baseline['rating'].values

In [62]:
test_baseline['baseline_rating'] = y_predicted
test_baseline.head()

,userId,movieId,rating,baseline_rating
0,11,39,4.5,3.639545
1,11,158,4.0,3.639545
2,11,380,5.0,3.639545
3,11,1862,1.0,3.639545
4,11,2124,3.5,3.639545


In [63]:
mae_baseline = mean_absolute_error(y, y_predicted)
rmse_baseline = RMSE(y, y_predicted)
precision_baseline = precision(y, y_predicted)

print('MAE: ', mae_baseline, '\nRMSE: ', rmse_baseline, '\nprecision: ', precision_baseline, '\nRunning time: ', t_baseline)

MAE:  0.8394352025099436 
RMSE:  1.04948481627348 
precision:  0.10759830712744191 
Running time:  1.1117608547210693


We will compare the performances of our models with these values. 

## Memory Based: user-based model

### Computing Similarity matrix

Now we will focus on the training set to build an user similarity matrix, using the adjusted cosine similarity.

Let's first adjust each row to remove the user bias.
For that, we first compute the average rating per user, and we substract this average to each rating to obtain a non biased rating column:

In [64]:
Training = training.toPandas()
Testing = test.toPandas()
training_ratings_mean = Training.groupby('userId', as_index=False).mean().rename(columns = {'rating': 'rating_mean'})[['userId','rating_mean']]
training_ratings_mean.head()

,userId,rating_mean
0,11,4.086957
1,50,4.000000
2,62,4.625000
3,82,4.148148
4,92,3.500000


In [65]:
training_adjusted_ratings = pd.merge(Training, training_ratings_mean, on='userId', how='left', sort=False)
training_adjusted_ratings['rating_adjusted'] = training_adjusted_ratings['rating'] - training_adjusted_ratings['rating_mean']
training_adjusted_ratings.head()

,userId,movieId,rating,rating_mean,rating_adjusted
0,11,110,4.0,4.086957,-0.086957
1,11,153,3.5,4.086957,-0.586957
2,11,231,4.0,4.086957,-0.086957
3,11,410,4.0,4.086957,-0.086957
4,11,480,5.0,4.086957,0.913043


In [66]:
training_adjusted_ratings = training_adjusted_ratings[['movieId', 'userId', 'rating_adjusted']]
training_adjusted_ratings.head()

,movieId,userId,rating_adjusted
0,110,11,-0.086957
1,153,11,-0.586957
2,231,11,-0.086957
3,410,11,-0.086957
4,480,11,0.913043


Here, the last column of the dataframe contains the adjusted rating on each user. We will then use this column to compute the similarity between each users using cosine similarity:
- firstly, select two distinct users
- secondly, isolate items that have been rated by both users
- thirdly, apply similarity computation, which is defined as the following:

$$sim(u,v) = \frac{\sum_{i \in I}R_{u,i} R_{v,i}}{\sqrt{\sum_{i \in I}R_{u,i}^2 R_{v,i}^2}}$$
Where:
- $R_{u,i}$ is the mean centered rating of item i given by user U
- $I$ is the set of items that are rated by u and v

In order to compute the similarity matrix, we will use the cosine_similarity method from the Sklearn library.

Let's first create our rating matrix. We put the userId in rows because we want to create an **user based** model. We can see that it is a very sparse matrix:

In [67]:
# First the original ratings matrix, used to compute the predicted ratings
training_rating_matrix = Training.pivot_table(values='rating', 
                                              index='userId', 
                                              columns='movieId')
training_rating_matrix.head()

movieId,8,12,15,18,23,26,36,39,49,59,...,120450,120813,120934,128960,129340,129909,129923,129925,130292,130342
userId,,,,,,,,,,,,,,,,,,,,,
11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
# Second the adusted rating matrix, used to compute the similarities between movies
training_adjusted_rating_matrix = training_adjusted_ratings.pivot_table(values='rating_adjusted', 
                                                                        index='userId', 
                                                                        columns='movieId')
training_adjusted_rating_matrix.head()

movieId,8,12,15,18,23,26,36,39,49,59,...,120450,120813,120934,128960,129340,129909,129923,129925,130292,130342
userId,,,,,,,,,,,,,,,,,,,,,
11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
# Creating a dummy matrix with 0 instead of Nan

dummy_training_adjusted_rating_matrix = training_adjusted_rating_matrix.copy().fillna(0)
dummy_training_adjusted_rating_matrix.head()

movieId,8,12,15,18,23,26,36,39,49,59,...,120450,120813,120934,128960,129340,129909,129923,129925,130292,130342
userId,,,,,,,,,,,,,,,,,,,,,
11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
62,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
82,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
92,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [70]:
# Importing the cosine similarity method from the Sklearn library
from sklearn.metrics.pairwise import cosine_similarity

# Computing the similarities
cosine_similarity = cosine_similarity(dummy_training_adjusted_rating_matrix, dummy_training_adjusted_rating_matrix)

In [71]:
# converting the cosine similarity into a similarity matrix
cosine_similarity = pd.DataFrame(cosine_similarity, 
                                 index=dummy_training_adjusted_rating_matrix.index, 
                                 columns=dummy_training_adjusted_rating_matrix.index)
cosine_similarity.head()

userId,11,50,62,82,92,114,125,128,159,164,...,138316,138331,138345,138367,138370,138379,138414,138423,138424,138448
userId,,,,,,,,,,,,,,,,,,,,,
11,1.000000,0.020519,0.0,-0.024813,0.00,0.0,0.0,0.087555,-2.724801e-18,0.012245,...,0.0000,0.099354,0.0,0.038657,0.081055,0.0,-0.037505,0.000000,0.099595,0.0
50,0.020519,1.000000,0.0,-0.111401,0.25,0.0,0.0,0.000000,0.000000e+00,0.000000,...,0.0000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
62,0.000000,0.000000,1.0,0.000000,0.00,0.0,0.0,0.000000,0.000000e+00,0.000000,...,-0.1066,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.307729,0.000000,0.0
82,-0.024813,-0.111401,0.0,1.000000,0.00,0.0,0.0,0.072500,0.000000e+00,0.132661,...,0.0000,0.000000,0.0,0.000000,0.081057,0.0,-0.016942,0.000000,0.000000,0.0
92,0.000000,0.250000,0.0,0.000000,1.00,0.0,0.0,0.000000,0.000000e+00,0.000000,...,0.0000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0


### Computing predicted ratings

Now we can complete the rating matrix by computing each missing rating using weighted mean ratings. 
More precisely, each missing entry is given by : 

$$ r_{u,i} = \frac{\sum_{i \in N}Similarity_{u,v} * r_{v,i}}{\sum_{j \in N}\mid\mid Similarity_{u,v} \mid\mid } $$

where $N$ is the set of users that have rated $i$.

In [74]:
import heapq
def user_based_collab(userId, movieId, k=10):
    
    # First check if the user is in the training set. Otherwise we do not have computed similarities for her.
    if userId in training_rating_matrix.index and movieId in training_rating_matrix:
        
        # get the similarities between the movie and the other movies
        user_similarities = cosine_similarity[userId]
        #print('User Similarities: ', user_similarities)
        
        # Get all the other users' ratings for this movie
        other_users_ratings = training_rating_matrix[movieId]
        #print('other users ratings: ', other_users_ratings)
        
        # Remove the NaN from the users' ratings and from the similarity vector
        nan_index = other_users_ratings[other_users_ratings.isnull()].index
        other_users_ratings = other_users_ratings.dropna()
        user_similarities = user_similarities.drop(nan_index)
        #print('DROPING NAN')
        #print('User Similarities: ', user_similarities)
        #print('other users ratings: ', other_users_ratings)
        
        # take k nearest neighbors
        k_index = user_similarities[user_similarities.isin(heapq.nlargest(k, user_similarities))].index
        other_users_ratings = other_users_ratings[k_index]
        user_similarities = user_similarities[k_index]
        #print('TAKING K NEAREST')
        #print('User Similarities: ', user_similarities)
        #print('other users ratings: ', other_users_ratings)
           
        
        # Compute the predicted rating
        s = 0
        for sim in user_similarities:
            s += abs(sim)
        if s > 0:
            return np.dot(user_similarities, other_users_ratings) / s
        else:
            return mu
    
        
    # If the user or the movie were not in the training set, return the average rating
    else:
        return mu

In [75]:
t0 = time.time()
predictions_user_based =[]
timecheck = 0
for row in Testing.index:
    if timecheck % 2000 ==0: ####################### A ENLEVER ##################
        print(timecheck / 178115)
        
    userId = Testing.loc[row, 'userId']
    movieId = Testing.loc[row, 'movieId']
    rating = user_based_collab(userId, movieId)
    #print(rating)
    predictions_user_based.append(rating)
    timecheck += 1
t_user_based = time.time() - t0
print ("Time of the training:", t_user_based)

0.0
0.011228700558627853
0.022457401117255707
0.03368610167588356
0.044914802234511414
0.05614350279313926
0.06737220335176712
0.07860090391039497
0.08982960446902283
0.10105830502765067
0.11228700558627852
0.12351570614490638
0.13474440670353424
0.1459731072621621
0.15720180782078993
0.16843050837941778
0.17965920893804566
0.1908879094966735
Time of the training: 246.2354609966278


In [76]:
Testing['User_based_predictions'] = predictions_user_based
Testing

,userId,movieId,rating,User_based_predictions
0,11,39,4.5,3.837469
1,11,158,4.0,2.658506
2,11,380,5.0,3.311402
3,11,1862,1.0,2.245200
4,11,2124,3.5,3.443820
...,...,...,...,...
35674,138414,6093,3.5,3.230565
35675,138414,85510,2.0,3.035770
35676,138414,91658,4.5,3.730125
35677,138414,103228,4.0,3.298367


In [77]:
y_1 = Testing['rating']
y_predicted_1 = Testing['User_based_predictions']

mae_user_based = mean_absolute_error(y_1, y_predicted_1)
rmse_user_based = RMSE(y_1, y_predicted_1)
precision_user_based = precision(y_1, y_predicted_1)                                     

print('MAE: ', mae_user_based, '\nRMSE: ', rmse_user_based, '\nprecision: ', precision_user_based, '\nRunning time: ', t_user_based)

MAE:  0.7530148505994723 
RMSE:  0.9874395250459447 
precision:  0.21819557723030353 
Running time:  246.2354609966278


Fortunately, we can observe an improvement in our metrics by ...
This model is more effective than the baseline model because it is more personalized. Indeed, it takes into account the users behaviors and their similarities. However, due to an important sparsity, this is not the best that can be done: the performances would be poorer if we had taken less popular movies with fewer ratings in our sample dataset.

In [78]:
################### A ENLEVER ####################

Testing.to_csv('Predition_userBased')

## Model Based: Matrix Factorization

### 1. Simple ALS method
We will first create a simple ALS method to fit the training set, without fitting the hyper parameters.
We start with a rank of 20, 20 iterations, $\lambda=0.01$ and we set the constraint of non-negativity to True. 

Let's first import the required packages:

In [79]:
from pyspark.ml.recommendation import ALS, ALSModel
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator

In [80]:
# Create ALS Model
SimpleAls = ALS(rank=20, maxIter=20, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop", nonnegative=True)

In [81]:
import time

t0 = time.time()
SimpleModel = SimpleAls.fit(training)
t_simpleALS = time.time() - t0
print ("Time of the training:", t_simpleALS)

Time of the training: 13.493462085723877


We can therefore use the simple model to compute the predictions:

In [82]:
# Define evaluator as RMSE
evaluator_rmse = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')
evaluator_mae = RegressionEvaluator(metricName='mae', labelCol='rating', predictionCol='prediction')

In [83]:
# Generate predictions and evaluate RMSE
predictions_training_1 = SimpleModel.transform(training)
predictions_test_1 = SimpleModel.transform(test)

rmse_training_1 = evaluator_rmse.evaluate(predictions_training_1)
rmse_test_1 = evaluator_rmse.evaluate(predictions_test_1)

mae_training_1 = evaluator_mae.evaluate(predictions_training_1)
mae_test_1 = evaluator_mae.evaluate(predictions_test_1)

In [84]:
predictions_test_1.show()

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|116259|   2366|   4.0|  2.553282|
| 23523|   2366|   2.0| 3.2879179|
|110203|   2366|   2.0| 4.0087166|
| 64989|   2366|   4.0|  4.045273|
|   970|   2366|   4.0| 3.0192065|
|105838|   2366|   3.0| 3.4491544|
|130389|   2366|   3.0|  2.779157|
|102687|   2366|   3.0| 4.9246736|
|137869|   2366|   4.0|  3.165233|
| 31653|   2366|   5.0| 2.7353745|
| 65459|   2366|   5.0| 3.0042324|
| 88488|   2366|   5.0| 3.5921257|
| 66819|   2366|   2.5|  3.690729|
| 55659|   2366|   5.0| 3.6960428|
| 18191|   2366|   3.5| 3.0290635|
|114509|   2366|   5.0|   4.21483|
| 23078|   2366|   5.0| 2.6479535|
| 64306|   2366|   4.0| 5.0989137|
|106273|   2366|   3.0|  3.652625|
|  9291|   2366|   5.0| 3.3707447|
+------+-------+------+----------+
only showing top 20 rows



In [85]:
predictions_test_1_pd = predictions_test_1.toPandas()
y_2 = predictions_test_1_pd['rating']
y_predicted_2 = predictions_test_1_pd['prediction']
precision_simpleALS = precision(y_2, y_predicted_2)  


print('For the training set:')
print('MAE: ', mae_training_1, '\nRMSE: ', rmse_training_1)
print()
print('For the test set:')
print('MAE: ', mae_test_1, '\nRMSE: ', rmse_test_1, '\nprecision:', precision_simpleALS)
print()
print('Running time', t_simpleALS)

For the training set:
MAE:  0.2546970871345407 
RMSE:  0.3885603529758982

For the test set:
MAE:  0.8552488790254889 
RMSE:  1.113330080336962 
precision: 0.19937413661864622

Running time 13.493462085723877


This model is quite simple, we did not try to fit the hyperparameters, namely the dimension of the latent vectors (rank of the matrix), the number of ALS iterations and the regularization parameter.
<br>Despite its simplicity, it is way better than the user-based model. The running is much lower which means that we can afford to often compute new predictions for a larger dataset.
<br><br>
We can now generate Top 10 user recommendations:

In [86]:
userRecs = SimpleModel.recommendForAllUsers(10).toPandas().set_index('userId')
userRecs.head()

,recommendations
userId,
4900,"[(1533, 7.49697208404541), (211, 7.39726638793..."
62680,"[(5272, 6.3474531173706055), (26222, 6.3337469..."
133590,"[(101360, 4.555708408355713), (6887, 4.3460507..."
471,"[(49, 6.474031448364258), (5883, 6.29868698120..."
35351,"[(27366, 7.5453410148620605), (6394, 6.3913321..."


### 2. Better ALS method: fitting the hyperparameters

One way to improve our previous model is to fit the hyperparameters to the model. For this, we are using the *pyspark.ml.tuning* library and its ParamGridBuilder function.

Here we are offering differents possibilites for the hyperparameters and the model will try them all to find the best one.

In [87]:
# Create ALS Model
FitALS = ALS(userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop", nonnegative=True)

In [88]:
# Tune model using ParamGridBuilder
param_grid = ParamGridBuilder().addGrid(FitALS.rank, [15,17,20]).addGrid(FitALS.maxIter, [15,20,25]).addGrid(FitALS.regParam, [.01,.05,.1]).build()

Let us use a cross-validation set up to train and tune our model. We will split the training set into a train set and a tune set, with ratio 0.8, 0.2 respectively:

In [89]:
# Build cross validation using TrainValidationSplit
tvs = TrainValidationSplit(estimator=FitALS, estimatorParamMaps=param_grid, evaluator=evaluator_rmse, trainRatio=0.8)

In [90]:
# Fit ALS model to train data
import time

t0 = time.time()
FitModel = tvs.fit(training)
t_fitALS = time.time() - t0
print ("Time of the training:", t_fitALS)

Time of the training: 628.3622989654541


Let's save the model:

In [92]:
#FitModel.save("FitModelALS")

In [93]:
# Extract the best model from the tuning of Hyperparameters
best_model = FitModel.bestModel

In [94]:
# Generate predictions and evaluate RMSE
predictions_training_2 = best_model.transform(training)
predictions_test_2 = best_model.transform(test)

rmse_training_2 = evaluator_rmse.evaluate(predictions_training_2)
rmse_test_2 = evaluator_rmse.evaluate(predictions_test_2)

mae_training_2 = evaluator_mae.evaluate(predictions_training_2)
mae_test_2 = evaluator_mae.evaluate(predictions_test_2)

We can print the metrics for this new model:

In [95]:
predictions_test_2_pd = predictions_test_2.toPandas()
y_3 = predictions_test_2_pd['rating']
y_predicted_3 = predictions_test_2_pd['prediction']
precision_fitALS = precision(y_3, y_predicted_3)  

print('Best hyperparameters found: ')
print('Rank:', best_model.rank)
print('MaxIter:', best_model._java_obj.parent().getMaxIter())
print('RegParam:', best_model._java_obj.parent().getRegParam())
print()
print('Metrics: ')
print('For the training set:')
print('MAE: ', mae_training_2, '\nRMSE: ', rmse_training_2)
print()
print('For the test set:')
print('MAE: ', mae_test_2, '\nRMSE: ', rmse_test_2, '\nprecision: ', precision_fitALS)
print()
print('Running time', t_fitALS)

Best hyperparameters found: 
Rank: 20
MaxIter: 25
RegParam: 0.1

Metrics: 
For the training set:
MAE:  0.4188332312069686 
RMSE:  0.548628358986449

For the test set:
MAE:  0.6943894470241447 
RMSE:  0.89518269400078 
precision:  0.24073186546756506

Running time 628.3622989654541


We see that the **error** is much lower than the previous one. Fitting the hyperparameters is effective, even for this bigger dataset.

Let's see how our model predict the actual ratings of the test set:

In [96]:
predictions_test_2.show()

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|116259|   2366|   4.0| 3.2593045|
| 23523|   2366|   2.0|  3.629482|
|110203|   2366|   2.0|  2.945938|
| 64989|   2366|   4.0| 3.3142715|
|   970|   2366|   4.0| 3.3248937|
|105838|   2366|   3.0| 3.5206316|
|130389|   2366|   3.0| 2.6987567|
|102687|   2366|   3.0|  2.613385|
|137869|   2366|   4.0| 3.5618176|
| 31653|   2366|   5.0| 3.2259493|
| 65459|   2366|   5.0|  3.160637|
| 88488|   2366|   5.0|  4.171378|
| 66819|   2366|   2.5| 3.1161258|
| 55659|   2366|   5.0| 3.8936477|
| 18191|   2366|   3.5| 3.1161618|
|114509|   2366|   5.0| 3.7652512|
| 23078|   2366|   5.0|  3.166909|
| 64306|   2366|   4.0| 4.1027346|
|106273|   2366|   3.0| 3.5051644|
|  9291|   2366|   5.0| 3.7690983|
+------+-------+------+----------+
only showing top 20 rows



We can now generate the top 10 recommendations for all users:

In [97]:
user_recs_fit = best_model.recommendForAllUsers(10).toPandas().set_index('userId')
user_recs_fit.head()

,recommendations
userId,
4900,"[(26225, 4.631032466888428), (5953, 4.58132839..."
62680,"[(104069, 5.350577354431152), (6511, 5.3115510..."
133590,"[(87308, 4.327284812927246), (6511, 4.27133893..."
471,"[(6511, 4.365274429321289), (2930, 4.327875614..."
35351,"[(25975, 4.728670120239258), (908, 4.616785049..."
